In [1]:
import os
import pandas as pd
import numpy as np
import re
#определяю регион страны
from countryinfo import CountryInfo

Гипотеза:  
Экономика государств с развитой государственной поддержкой инновационных предприятий (инновационной системы) растёт интенсивнее.
Для этого, выбираю следующие показатели:  

___1.1.	Political environment (интегральный покозатель)___  
1.1.1.	Political stability and absence of violence/terrorism  
1.1.2.	Government effectiveness  
1.1.3.	Press freedom  

___1.2.	Regulatory environment (интегральный покозатель)___  
1.2.1.	Regulatory quality  
1.2.2.	Rule of law  
1.2.3.	Cost of redundancy dismissal  

___1.3.	Business environment (интегральный покозатель)___  
1.3.1.	Ease of starting a business  
1.3.2.	Ease of resolving insolvency  
1.3.3.	Ease of paying taxes  

2.1.1.	Expenditure on education  
2.1.2.	Public expenditure on education per pupil  

2.1.5.	Pupil-teacher ratio, secondary  

2.2.1.	Tertiary enrolment  
2.2.2.	Graduates in science and engineering  

2.3.1.	Researchers  
2.3.2.	Gross expenditure on R&D (GERD)  

3.1.3.	Government's online service  

3.2.1.	Electricity output  
3.2.3.	Logistics performance  

3.3.2.	Environmental performance  


___4.1.	Credit (интегральный покозатель)___  
4.1.1.	Ease of getting credit  
4.1.2.	Domestic credit to private sector  
4.1.3.	Microfinance institutions' gross loan portfolio  

4.2.1.	Ease of protecting investors  

4.2.4.	Venture capital deals  

4.3.2.	Market access for non-agricultural exports  
4.3.3.	Intensity of local competition  

5.1.2.	Firms offering formal training  

___5.2.	Innovation linkages (интегральный покозатель)___   
5.2.1.	University/industry research collaboration  
5.2.2.	State of cluster development  
5.2.3.	GERD financed by abroad  
5.2.4.	Joint venture/strategic alliance deals  
5.2.5.	Patent families filed in at least three offices  

5.3.1.	Royalties and license fees payments (% of service imports)  
5.3.4.	Foreign direct investment net inflows  

6.1.2.	Patent Cooperation Treaty resident applications  

6.3.4.	Foreign direct investment net outflows

Я выбрал 37 показателей, которые могут характеризовать государственную поддержку и 5 интегральных позателей (вероятно они могут быть полезны для свёртки)

In [2]:
data_dir = 'C:\\Users\\dserp\\ML\\Degree_ML\\full'
file_list = os.listdir(data_dir)

## Формирую датасет

In [3]:
data ={} #словарь датасетов с показателми

In [4]:
df13 = pd.read_csv(os.path.join(data_dir, file_list[0]))
df13.head()

,,.1,Rank Indicators,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,...,United Kingdom.4,United States of America.4,Uruguay.4,Uzbekistan.4,"Venezuela, Bolivarian Republic of.4",Viet Nam.4,Yemen.4,Zambia.4,Zimbabwe.4,Unnamed: 752
0,Indicator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Global Innovation Index,,,93.0,138.0,135.0,56.0,59.0,19.0,23.0,...,Strength,,,Weakness,,,Weakness,,Weakness,NaN
2,Innovation Efficiency Index,,,129.0,141.0,22.0,20.0,42.0,116.0,98.0,...,Weakness,Weakness,,Weakness,Strength,Strength,,Strength,Strength,NaN
3,Innovation Input,,,77.0,112.0,140.0,78.0,71.0,11.0,17.0,...,Strength,Strength,,,Weakness,,Weakness,,Weakness,NaN
4,Innovation Output,,,118.0,141.0,117.0,43.0,47.0,32.0,27.0,...,Strength,,,Weakness,,,,,,NaN


In [5]:
df13.describe().T

,count,mean,std,min,25%,50%,75%,max
Albania,109.0,78.110092,30.394425,5.0,63.000,81.00,94.000,136.0
Algeria,102.0,99.401961,28.680990,7.0,80.500,105.00,120.750,141.0
Angola,94.0,110.287234,31.836823,18.0,97.000,122.00,134.000,142.0
Argentina,109.0,64.844037,31.064203,3.0,43.000,59.00,87.000,127.0
Armenia,108.0,63.277778,28.223353,2.0,46.500,61.00,82.500,129.0
...,...,...,...,...,...,...,...,...
Viet Nam.2,71.0,58.794366,188.403940,-0.6,1.250,5.20,27.500,1091.6
Yemen.2,62.0,25.967742,69.178140,-5.8,0.300,2.75,16.775,374.6
Zambia.2,64.0,44.145313,141.779670,-0.7,0.475,3.65,26.475,874.5
Zimbabwe.2,61.0,50.022951,162.991837,-1.9,0.300,2.40,38.100,1022.2


Предобработка

In [6]:
#Для дальнейшей обработки я делаю лист названий колонок
col_list = list(df13)
#Переименовываю столбцы 
df13.rename(columns={col_list[0] : re.sub('\D','', file_list[0]),col_list[1] : 'Indicator', col_list[2] : 'subindicator', col_list[752] : 'to_del'}, inplace=True)

#удаляю строку 0 и 5 - они содержат только Nan (видимо для красоты таблички, для открытия в Excel)
df13.drop([0, 5], inplace = True)

#удаляю столбец 'to_del' - пустой
df13.drop('to_del', axis = 1, inplace = True)

#заполняю пропуски нулями
df13.fillna(0, inplace = True)

#в датафрейме есть значения '  ' - несколько пробелов, заменяю их на None
space = df13['United States of America.4'][1]
df13.replace(space, np.nan, inplace = True)

#отсекаю всё, лишнее
df13 = df13.loc[:, :'Zimbabwe']

In [7]:
# в gii будет хранитсья все gii c 2013 по 2018 г.
gii = df13.loc[:1, 'Albania':'Zimbabwe']
gii.index.values[0] = re.sub('\D','',file_list[0])
gii.head()

,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahrain,Bangladesh,...,United Arab Emirates,United Kingdom,United States of America,Uruguay,Uzbekistan,"Venezuela, Bolivarian Republic of",Viet Nam,Yemen,Zambia,Zimbabwe
2013,93.0,138.0,135.0,56.0,59.0,19.0,23.0,105.0,67.0,130.0,...,38.0,3.0,5.0,52.0,133.0,114.0,76.0,142.0,118.0,132.0


In [8]:
# отобранные показатели (37 шт)
list_fe = ['Political stability and absence of violence/terrorism', 'Government effectiveness', 'Press freedom','Regulatory quality', 'Rule of law', 'Cost of redundancy dismissal', 'Ease of starting a business', 'Ease of resolving insolvency', 'Ease of paying taxes', 'Expenditure on education', 'Public expenditure on education per pupil', 'Pupil-teacher ratio, secondary', 'Tertiary enrolment','Graduates in science and engineering','Researchers','Gross expenditure on R&D (GERD)',"Government's online service",'Electricity output','Logistics performance','Environmental performance','Ease of getting credit','Domestic credit to private sector',"Microfinance institutions' gross loan portfolio", 'Ease of protecting investors', 'Venture capital deals', 'Market access for non-agricultural exports', 'Intensity of local competition', 'Firms offering formal training', 'University/industry research collaboration', 'State of cluster development', 'GERD financed by abroad', 'Joint venture/strategic alliance deals', 'Patent families filed in at least three offices', 'Royalties and license fees payments (% of service imports)','Foreign direct investment net inflows', 'Patent Cooperation Treaty resident applications', 'Foreign direct investment net outflows']
list_intfe = ['Political environment', 'Regulatory environment', 'Business environment', 'Credit']

In [9]:
#дтатафрейм отобранных показателей
indicators = pd.DataFrame(data = None, columns = list(df13))

In [10]:
for i in list_fe:
    indicators = pd.concat([indicators, df13.loc[df13['Indicator']== i]])
indicators.drop('subindicator', axis = 1, inplace = True)

In [11]:
#добавляю показатели в датасет
data[re.sub('\D','', file_list[0])] = indicators

In [12]:
err_country = []
region = pd.DataFrame({'country': list(indicators)[3:], 'region': None})
for i in range(len(region.country)):
    try:
        region.region[i] = CountryInfo(region.country[i]).subregion()
    except:
        err_country.append(region.country[i])

In [13]:
print(err_country, '\n')
print('Количество нераспознанных стран:', len(err_country))

['Bolivia, Plurinational State of', 'Brunei Darussalam', 'Cote dIvoire', 'Gambia', 'Hong Kong (China)', 'Iran, Islamic Republic of', 'Korea, Republic of', 'Moldova, Republic of', 'Montenegro', 'Taiwan, Province of China', 'Tanzania, United Republic of', 'The Former Yugoslav Republic (FYR) of Macedonia', 'Venezuela, Bolivarian Republic of', 'Viet Nam'] 

Количество нераспознанных стран: 14


In [14]:
#Заполняю в ручную
region.loc[region.country == err_country[0],['region']] = 'South America' #Bolivia
region.loc[region.country == err_country[1],['region']] = 'South-Eastern Asia' #Brunei
region.loc[region.country == err_country[2],['region']] = 'Western Africa' #Ivory Coast
region.loc[region.country == err_country[3],['region']] = 'Western Africa' #Republic of The Gambia
region.loc[region.country == err_country[4],['region']] = 'Eastern Asia' #Hong Kong
region.loc[region.country == err_country[5],['region']] = 'Southern Asia' #Iran
region.loc[region.country == err_country[6],['region']] = 'Eastern Asia' #Republic of Korea
region.loc[region.country == err_country[7],['region']] = 'Eastern Europe' #Moldova
region.loc[region.country == err_country[8],['region']] = 'Southern Europe' #Montenegro - не знает CountryInfo, я испольовал соседа - Bosnia-Herzegovina
region.loc[region.country == err_country[9],['region']] = 'Eastern Asia' #Taiwan
region.loc[region.country == err_country[10],['region']] = 'Eastern Africa' #Tanzania
region.loc[region.country == err_country[11],['region']] = 'Eastern Europe'#The Former Yugoslav Republic (FYR) of Macedonia - не знает CountryInfo, я испольовал Serbia
region.loc[region.country == err_country[12],['region']] ='South America' #Venezuela
region.loc[region.country == err_country[13],['region']] = 'South-Eastern Asia'#VietNam

In [15]:
region.head(5)

,country,region
0,Algeria,Northern Africa
1,Angola,Middle Africa
2,Argentina,South America
3,Armenia,Western Asia
4,Australia,Australia and New Zealand


In [16]:
list(indicators)

['2013',
 'Indicator',
 'Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bolivia, Plurinational State of',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Cote dIvoire',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Guyana',
 'Honduras',
 'Hong Kong (China)',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran, Islamic Republic of',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Korea, Republic of',
 'Kuwait',
 

In [17]:
for file in file_list[1:]:
    df_temp = pd.read_csv(os.path.join(data_dir, file))
        
    #Переименовываю столбцы 
    #df_temp.rename(columns={col_list[0] : re.sub('\D','', file),col_list[1] : 'Indicator', col_list[2] : 'subindicator', col_list[752] : 'to_del'}, inplace=True)
    df_temp.rename(columns={col_list[0] : re.sub('\D','', file),col_list[1] : 'Indicator', col_list[2] : 'subindicator'}, inplace=True)
    
    #удаляю строку 0 и 5 - они содержат только Nan (видимо для красоты таблички, для открытия в Excel)
    df_temp.drop([0, 5], inplace = True)

    #удаляю столбец 'to_del' - пустой
    #df_temp.drop('to_del', axis = 1, inplace = True)

    #заполняю пропуски нулями
    df_temp.fillna(0, inplace = True)

    #в датафрейме есть значения '  ' - несколько пробелов, заменяю их на None
    df_temp.replace(space, np.nan, inplace = True)
    
    #отсекаю всё, лишнее
    df_temp = df_temp.loc[:, :'Zimbabwe']
    
    gii_temp = df_temp.loc[:1, 'Albania':'Zimbabwe']
    gii_temp.index.values[0] = re.sub('\D','',file)
    gii = pd.concat([gii, gii_temp])
    
    
    indicators_temp = pd.DataFrame(data = None, columns = list(df_temp))
    for i in list_fe:
        indicators_temp = pd.concat([indicators_temp, df_temp.loc[df_temp['Indicator']== i]])
    indicators_temp.drop('subindicator', axis = 1, inplace = True)
    
    #добавляю показатели в датасет
    data[re.sub('\D','', file)] = indicators_temp


D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [18]:
data.keys()

dict_keys(['2013', '2014', '2015', '2016', '2017', '2018', '2019'])

In [22]:
list(data['2015'])

['2015',
 'Indicator',
 'Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia, Plurinational State of',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo, Democratic Republic of the',
 'Cook Islands',
 'Costa Rica',
 'Cote dIvoire',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Faroe Islands',
